# Bayesian optimization

This tutorial is an introduction to the syntax used by the optimizer, as well as the principles of Bayesian optimization in general.

We'll start by minimizing the Rastrigin function in one dimension, which looks like this:

In [ ]:
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt

from bloptools import test_functions

x = np.linspace(-4, 4, 256)

plt.plot(x, test_functions.rastrigin(x), c="b")

There are several things that our agent will need. The first ingredient is some degrees of freedom (these are always `ophyd` devices) which the agent will move around to different inputs within each DOF's bounds (the second ingredient). We define these here:

In [ ]:
import bloptools

dofs = bloptools.devices.dummy_dofs(n=1)  # an ophyd device that we can read/set

bounds = [(-4.0, 4.0)]  # one set of bounds per dof

This degree of freedom will move around a variable called `x1`. The agent automatically samples at different inputs, but we often need some post-processing after data collection. In this case, we need to give the agent a way to compute the Rastrigin function. We accomplish this with a digestion function, which always takes `(db, uid)` as an input. For each entry, we compute the function:


In [ ]:
def digestion(db, uid):
    products = db[uid].table()

    for index, entry in products.iterrows():
        products.loc[index, "rastrigin"] = test_functions.rastrigin(entry.x1)

    return products

The next ingredient is a task, which gives the agent something to do. We want it to minimize the Rastrigin function, so we make a task that will try to minimize the output of the digestion function called "rastrigin".

In [ ]:
from bloptools.tasks import Task

task = Task(key="rastrigin", kind="min")

In [ ]:
os.environ["BLUESKY_DEBUG_CALLBACKS"] = "True"

import bluesky
print(bluesky.__version__)

Combining all of these with a databroker instance, we can make an agent:

In [ ]:
%run -i ../../../examples/prepare_bluesky.py # prepare the bluesky environment

agent = bloptools.bayesian.Agent(
    active_dofs=dofs,
    passive_dofs=[],
    active_dof_bounds=bounds,
    tasks=[task],
    digestion=digestion,
    db=db,
)

RE(agent.initialize(acqf="qr", n_init=12))

We initialized the GP with the "quasi-random" strategy, as it doesn't require any prior data. We can view the state of the optimizer's posterior of the tasks over the input parameters:

In [ ]:
# what are the points?

agent.plot_tasks()

Note that the value of the fitness is the negative value of the function: we always want to maximize the fitness of the tasks.

An important concept in Bayesian optimization is the acquisition function, which is how the agent decides where to sample next. Under the hood, the agent will see what inputs maximize the acquisition function to make its decision.

We can see what the agent is thinking by asking it to plot a few different acquisition functions in its current state.

In [ ]:
# helper function to list acquisition functions

agent.plot_acquisition(strategy=["ei", "pi", "ucb"])

Let's tell the agent to learn a little bit more. We just have to tell it what acquisition function to use (by passing a `strategy`) and how many iterations we'd like it to perform (by passing `n_iter`).

In [ ]:
RE(agent.learn(acqf="ei", n_iter=8))
agent.plot_tasks()